In [1]:
from model import HancyModel
from dataloader import NlpDataset, data_processing
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import functional as F
import torch.nn as nn
from argparse import ArgumentParser
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import random_split
import torch
import pickle

from utils import processLabels
import pytorch_lightning as pl

/home/sam/.local/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(
/home/sam/.local/lib/python3.8/site-packages/torchaudio/functional.py:317: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [2]:
def checkpoint_callback(args):
    return ModelCheckpoint(
        filepath=args.save_model_path,
        save_top_k=True,
        verbose=True,
        monitor="val_loss",
        mode="min",
        prefix="",
    )


class SpeechRecog(pl.LightningModule):
    def __init__(self, model, args):
        super().__init__()
        self.model = model
        self.criterion = nn.CTCLoss(blank=28, zero_infinity=True)
        self.args = args

    def configure_optimizers(self):
        self.optimizer = optim.AdamW(self.model.parameters(), 0.01)
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode="min", factor=0.50, patience=6
        )
        self.scheduler = {
            'scheduler': lr_scheduler, 
            'reduce_on_plateau': True,
            # val_checkpoint_on is val_loss passed in as checkpoint_on
            'monitor': 'val_checkpoint_on'
        }
        return [self.optimizer], [self.scheduler]

    def forward(self, x, hidden):
        return self.model(x, hidden)

    def step(self, batch):
        spectrograms, labels, input_lengths, label_lengths = batch
        bs = spectrograms.shape[0]
        hidden = self.model._init_hidden(bs)
        hn, c0 = hidden[0].to(self.device), hidden[1].to(self.device)
        output, _ = self(spectrograms, (hn, c0))
        output = F.log_softmax(output, dim=2)
        loss = self.criterion(output, labels, input_lengths, label_lengths)
        return loss

    def training_step(self, train_batch, batch_idx):
        loss = self.step(train_batch)
        logs = {"loss": loss, "lr": self.optimizer.param_groups[0]["lr"]}
        return {"loss": loss, "log": logs}

    def validation_step(self, val_batch, batch_idx):
        loss = self.step(val_batch)
        return {"val_loss": loss}

In [3]:
class Args():
    def __init__(self):
        self.load_x = "/home/sam/Downloads/firstzipdata.pt"
        self.load_y = "/home/sam/Downloads/firstziplabel.txt"
        self.save_model_path = "checkpoint"
        self.load_model_from = "checkpoint"

args = Args()

print("Loading x label")
x = torch.load(args.load_x)
print("Loaded all tensor")
print("Loading y label")
y = processLabels(pickle.load(open(args.load_y, "rb")))
print("Loaded all label")

Loading x label
Loaded all tensor
Loading y label
Loaded all label


In [4]:

dataset = NlpDataset(x, y)
tt = int(len(dataset) * 0.8)
tl = len(dataset) - tt
train, val = random_split(dataset, [tt, tl])
train_loader = DataLoader(
    dataset=train, collate_fn=data_processing
)
val_loader = DataLoader(dataset=val, collate_fn=data_processing)

model = HancyModel()
speechmodule = SpeechRecog(model, args)
trainer = pl.Trainer(
    max_epochs=2,
    gpus=1,
    num_nodes=1,
    distributed_backend=None,
    gradient_clip_val=1.0,
    val_check_interval=0.25,
    checkpoint_callback=checkpoint_callback(args),
    # resume_from_checkpoint=args.resume_from_checkpoint,
)
trainer.fit(speechmodule, train_loader, val_loader)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | model     | HancyModel | 355 K 
1 | criterion | CTCLoss    | 0     
-----------------------------------------
355 K     Trainable params
0         Non-trainable params
355 K     Total params
/home/sam/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/sam/.local/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/home/sam/.local/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore


ModuleAttributeError: 'HancyModel' object has no attribute '_init_hidden'